In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
base_path = '/content/drive/MyDrive/ICBT BSc/RSDetector/'
recognition_dataset_path = base_path + 'dataset_images'
video_path = base_path + 'dataset_videos'
sign_info_path = base_path + 'sign_data.csv'

In [ ]:
#!unzip '/content/drive/MyDrive/ICBT BSc/RSDetector/Dataset.zip' -d '/content/drive/MyDrive/ICBT BSc/RSDetector/'

In [ ]:
sign_info = pd.read_csv(sign_info_path)

In [ ]:
sign_info

# Dataset

## Detection dataset

In [ ]:
!pip install roboflow

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="idKFK8PupLHRjgqWDlCC")
project = rf.workspace("rsdetector").project("roadsign-frnif")
dataset = project.version(4).download("yolov8")

# Model Training

## Detection model training

In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics

In [ ]:
from ultralytics import settings

settings.update({'runs_dir': '/content/drive/MyDrive/ICBT BSc/RSDetector/model'})

In [ ]:
# segementation
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n-seg.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data= "{}/data.yaml".format(dataset.location), epochs=100, imgsz=640)

# Final Model

## Detection final model

In [ ]:
from ultralytics import YOLO
final_model = YOLO('/content/drive/MyDrive/ICBT BSc/RSDetector/model/segment/train13/weights/best.pt')
names = final_model.names


# Prediction

## Image Test

In [ ]:
nopark = '/content/drive/MyDrive/ICBT BSc/RSDetector/Slide2.PNG'

In [ ]:
from PIL import Image
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

results = final_model.predict(nopark,conf=0.6)  # results list
Image.fromarray(results[0].plot(labels=True)[...,::-1])


In [ ]:
round(float(results[0].boxes.conf),2)

for r in results:
  for k in r.boxes:
    print(k.cls)
    print(k.conf)


tensor([6.], device='cuda:0')
tensor([0.9330], device='cuda:0')


In [ ]:
bboxes = results[0].boxes.xyxy[0].cpu().numpy()
image = cv2.imread(nopark)

# Crop the image using the bounding box coordinates
x1, y1, x2, y2 = bboxes.astype(int)

cropped_image = image[y1:y2, x1:x2]

# Save the cropped images
cv2.imwrite(f'cropped_image.jpg', cropped_image)

True

## Video Test

In [ ]:
def crop_cls_image(x1, y1, x2, y2,resized_frame):
  # Calculate the center of the bounding box
  center_x = (x1 + x2) // 2
  center_y = (y1 + y2) // 2

  # Calculate the new cropping coordinates to ensure a 224x224 crop
  crop_x1 = max(center_x - crop_size // 2, 0)
  crop_x2 = min(crop_x1 + crop_size, resized_frame.shape[1])
  crop_y1 = max(center_y - crop_size // 2, 0)
  crop_y2 = min(crop_y1 + crop_size, resized_frame.shape[0])

  # Crop the region, including background if needed, to get a 224x224 image
  cropped_region = resized_frame[crop_y1:crop_y2, crop_x1:crop_x2]

  # Resize the cropped region to 224x224
  cropped_image = cv2.resize(cropped_region, (crop_size, crop_size))

  return cropped_image

def detection(frame,prev_cls):

  resized_frame = cv2.resize(frame, (640, 640))
  results = final_model(resized_frame,conf=0.95)

  for r in results:
    if r:
      bboxes = r.boxes.xyxy[0].cpu().numpy()
      x1, y1, x2, y2 = bboxes.astype(int)
      for c in r.boxes:
        conf =  round(float(c.conf),2)
        cls =  names[int(c.cls)]
        if prev_cls != cls:
          prev_cls = cls
          return (prev_cls,conf, crop_cls_image(x1, y1, x2, y2,resized_frame))
    return (None,None,None)

In [ ]:
import cv2
import numpy as np

source = '/content/drive/MyDrive/ICBT BSc/RSDetector/dataset_videos/4-1.mp4'
cap = cv2.VideoCapture(source)

# 10 frames per second
skip_factor = 10
counter = 0

generate_detection_video = False
annotated_frames = []

cropped_images = {}
cls = None
crop_size = 64


while cap.isOpened():
    success, frame = cap.read()

    if not success:
        print("Finished reading the video.")
        break

    if counter % skip_factor == 0:
       cls, conf, detected_image = detection(frame,cls)
       cropped_images[cls] = (conf,detected_image)



       if generate_detection_video:
          annotated_frame = results[0].plot(labels=False)
          annotated_frames.append(annotated_frame)
    counter += 1

cap.release()

In [ ]:
cv2.imwrite('cropped_image {}.jpg'.format(1), cropped_images['2'][1])

In [ ]:

cv2.imwrite('cropped_image {}.jpg'.format(1), cv2.resize(cropped_images['4'][1], (224, 224)) )

True

In [ ]:
cropped_images

In [ ]:
len(cropped_images['6'])

### save video

In [ ]:
import cv2
import numpy as np

# Define the output video filename and codec
output_filename = 'annotated_video.avi'  # Change the filename and extension as needed
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can change the codec as needed

# Get the dimensions of the first frame to set up the VideoWriter
frame_height, frame_width, _ = annotated_frames[0].shape
frame_rate = 30  # Adjust the frame rate as needed

# Create the VideoWriter object
out = cv2.VideoWriter(output_filename, fourcc, frame_rate, (frame_width, frame_height))

# Loop through the annotated frames and write them to the video
for frame in annotated_frames:
    # Write the frame to the video
    out.write(frame)

# Release the video writer
out.release()

print("Annotated video saved as", output_filename)


Annotated video saved as annotated_video.avi
